# Trabajo Practico 3 - Finanzas Computacionales (UdeSA)
## Bautista Remondino - Santiago Pini - Pedro Gallo - Ignacio Bidarte


# Ejercicio 1

Empezamos con el metodo explicito, este primer bloque muestra se configuro la grilla de $x$ e $y$, $\Delta \tau$ y $\lambda$ estable


In [14]:
import numpy as np
import math
import matplotlib.pyplot as plt

# Parámetros del problema
K = 100.0
r = 0.05
sigma = 0.20
T = 1.0

grilla_x = np.linspace(10, 200, 100)
grilla_t = np.linspace(0, T, 100)

S_min, S_max = 0.1*K, 4.0*K  
x_min, x_max = np.log(S_min), np.log(S_max)

Nx  = 400                              # num de intervalos en x
dx  = (x_max - x_min)/Nx
x   = np.linspace(x_min, x_max, Nx+1)

tau_max = 0.5 * sigma**2 * T

# Para el EXPÍCITO necesito λ = Δτ / (Δx)^2 ≤ 0.5  → elijo λ=0.45
lam   = 0.45
dtau  = lam * dx**2
Nt    = int(np.ceil(tau_max / dtau))   # cantidad de pasos de tiempo
dtau  = tau_max / Nt                   
lam   = dtau / dx**2                   # λ final (≤ 0.5 garantizado)
tau   = np.linspace(0.0, tau_max, Nt+1)


### Condiciones iniciales y de frontera

In [15]:
# Condición inicial: valor de la opción al vencimiento (t=T, τ=0)
y = np.exp(-r*T) * np.maximum(np.exp(x) - K, 0)

# Condición de frontera izquierda: S → 0  (x → -∞ o x_min)
# Notamos que es una frontera constante, no depende del tiempo por lo cual la podemos dejar fija.
def frontera_izquierda(tau):
    return 0.0

# Condición de frontera derecha: S → ∞  (x → +∞ o x_max)
def frontera_derecha(tau):
    return ( np.exp( -r*(T - 2.0*tau/sigma**2) )
             * np.exp( x_max - (2.0*r/sigma**2 - 1.0)*tau )
             - K * np.exp(-r*T) )


### Metodo explícito

Partimos de la ecuación del calor:

$$
\frac{\partial y}{\partial t} = \frac{\partial^2 y}{\partial x^2}
$$

Reemplazando por derivadas centrada y adelantada:

$$
\frac{y(x,t+1) - y(x,t)}{\Delta t} = \frac{y(x+1,t) - 2y(x,t) + y(x-1,t)}{\Delta x^2}
$$

Buscamos aislar $ y(x,t+1) $, por tanto multiplicamos a ambos lados $\Delta t$:

$$
y(x,t+1) - y(x,t) = \frac{\Delta t}{\Delta x^2} \left[ y(x+1,t) - 2y(x,t) + y(x-1,t) \right]
$$

Definimos $ \lambda = \frac{\Delta t}{\Delta x^2} $, entonces:

$$
y(x,t+1) = y(x,t) + \lambda \left[ y(x+1,t) - 2y(x,t) + y(x-1,t) \right]
$$

$$
y(x,t+1) = \lambda y(x+1,t) + (1+2\lambda) y(x,t) + \lambda y(x-1,t) 
$$

In [4]:
for n in range(Nt):
    y_new = y.copy()
    y_new[1:-1] = y[1:-1] + lam * (y[2:] - 2*y[1:-1] + y[:-2])
    # fronteras
    y_new[0]  = frontera_izquierda(tau[n+1])
    y_new[-1] = frontera_derecha(tau[n+1])
    y = y_new


# Tomamos un precio en especial y chequeamos el resultado numérico

shift = (2.0*r/sigma**2 - 1.0) * tau_max
S_grid = np.exp( x - shift )          # mapea x → S en t=0
V_num = y                             # en t=0 el factor exponencial es 1, luego V = y

S0 = K
idx = np.argmin( np.abs(S_grid - S0) )
V_S0 = V_num[idx]
print(f"Precio numérico explícito para S0={S0}, t=0: {V_S0}")

Precio numérico explícito para S0=100.0, t=0: 10.49231204305964


### Metodo Implícito

Nuevamente partimos de la ecuación del calor:

$$
\frac{\partial y}{\partial t} = \frac{\partial^2 y}{\partial x^2}
$$

Reemplazando por derivadas centrada y adelantada:

$$
\frac{y(x,t+1) - y(x,t)}{\Delta t} = \frac{y(x+1,t+1) - 2y(x,t+1) + y(x-1,t+1)}{\Delta x^2}
$$

Multiplicamos por $\Delta t$ y mandamos todos los terminos con $t+1$ para el otro lado:

$$
y(x,t+1) - y(x,t) = \frac{\Delta t}{\Delta x^2} \left[ y(x+1,t+1) - 2y(x,t+1) + y(x-1,t+1) \right]
$$

Definimos $ \lambda = \frac{\Delta t}{\Delta x^2} $, entonces:

$$
y(x,t) = y(x,t+1) - \lambda \left[ y(x+1,t+1) - 2y(x,t+1) + y(x-1,t+1) \right]
$$

$$
y(x,t) = - \lambda y(x+1,t+1) + (1+2\lambda) y(x,t+1) - \lambda y(x-1,t+1)
$$

### Matricialmente podriamos pensar este problema como:

Si definís el vector de incógnitas en los puntos interiores del dominio espacial:


$$
\mathbf{y(x,t+1)} =
\begin{bmatrix}
y(x,t+1) \\
y(x+1,t+1) \\
\vdots \\
y(N-1, t+1)
\end{bmatrix},
\quad
\mathbf{y(x,t)} =
\begin{bmatrix}
y(x,t) \\
y(x+1,t) \\
\vdots \\
y(N-1, t)
\end{bmatrix}.
$$


Entonces, el sistema de ecuaciones del método implícito se puede escribir como:

$$
A\,y(x, t + 1) = y(x, t) + d(x, t),
$$

donde la matriz \(A\) tiene estructura tridiagonal:

$$
A =
\begin{bmatrix}
1 + 2\lambda & -\lambda & 0 & \dots & 0 \\
-\lambda & 1 + 2\lambda & -\lambda & \ddots & \vdots \\
0 & \ddots & \ddots & \ddots & 0 \\
\vdots & \ddots & -\lambda & 1 + 2\lambda & -\lambda \\
0 & \dots & 0 & -\lambda & 1 + 2\lambda
\end{bmatrix}
\in \mathbb{R}^{(N-1)\times(N-1)}.
$$

y el vector \(d(x,t)\) incorpora las condiciones de frontera que entran solo en la primera y última ecuación interior:


$$
\mathbf{d}^{\,n+1} =
\begin{bmatrix}
\lambda\,y(0,t+1) \\
0 \\
\vdots \\
0 \\
\lambda\,y(N,n-1)
\end{bmatrix}.
$$

### Armado de la matriz A
Se reutilizan las condiciones iniciales y de frontera, como tambien las variables de $\Delta \tau$, la grilla de $x$ e $y$, etc.

In [12]:
main_diag = (1 + 2*lam) * np.ones(Nx-1) #vector con la diagonal principal
off_diag  = -lam * np.ones(Nx-2)         #vector con las diagonales secundarias de tamaño n-2
A = np.diag(main_diag) + np.diag(off_diag, 1) + np.diag(off_diag, -1) #compila las diagolanales en una matriz


for n in range(Nt):
    yL = frontera_izquierda(tau[n+1])  
    yR = frontera_derecha(tau[n+1])    

    # Construimos el vector d del lado derecho
    d = np.zeros(Nx-1)
    d[0]  = lam * yL
    d[-1] = lam * yR

    rhs = y[1:-1] + d
    y_interior = np.linalg.solve(A, rhs) # Resolvemos el sistema lineal usando metodos de Numpy que son menos costosos computacionalmente que aplicar la inversa directamente

    # Actualizamos el vector completo
    y[0]      = yL
    y[1:-1]   = y_interior
    y[-1]     = yR


# Mismo metodo que en el explícito para obtener el precio en S0=K

shift = (2.0*r/sigma**2 - 1.0) * tau_max   # tau_max = (σ²/2)T
S_grid = np.exp(x - shift)                  # S asociado a cada nodo x en t=0

# y contiene la solución final en τ=τ_max (t=0)
V_num = y.copy()                            # V(S,0) = y(x, τ_max)

S0 = K
idx = np.argmin(np.abs(S_grid - S0))
V_S0_nearest = V_num[idx]
print(f"[Implícito] Precio numérico (nearest) para S0={S0}: {V_S0_nearest:.6f}")

[Implícito] Precio numérico (nearest) para S0=100.0: 10.488893


### Metodo de Crank-Nicolson
Consiste en tomar un promedio de ambos esquemas (implícito y explícito):

$$
\frac{\partial y}{\partial t} = \frac{\partial^2 y}{\partial x^2}
$$

$$
\frac{y(x,t+1) - y(x,t)}{\Delta t} = \frac{1}{2\Delta x^2} \left[ [ y(x+1,t) - 2y(x,t) + y(x-1,t) ] + [ y(x+1, t+1) - 2y(x,t+1) + y(x-1,t+1)] \right ]
$$

Pasamos el $\Delta \tau$ del otro lado:

$$
\frac{y(x,t+1) - y(x,t)}{\Delta t} = \frac{1}{2\Delta x^2} \left[ [ y(x+1,t) - 2y(x,t) + y(x-1,t) ] + [ y(x+1, t+1) - 2y(x,t+1) + y(x-1,t+1)] \right ]
$$

$$
y(x,t+1) - y(x,t) = \frac{\Delta \tau}{2\Delta x^2} \left[ [ y(x+1,t) - 2y(x,t) + y(x-1,t) ] + [ y(x+1, t+1) - 2y(x,t+1) + y(x-1,t+1)] \right ]
$$

Conocemos la definición de $\lambda$:

$$
y(x,t+1) - y(x,t) = \lambda \left[ [ y(x+1,t) - 2y(x,t) + y(x-1,t) ] + [ y(x+1, t+1) - 2y(x,t+1) + y(x-1,t+1)] \right ]
$$

Despejamos de un lado los $t+1$ y finalmente nos queda:

$$
\frac{-\lambda}{2} y(x-1,t+1) + (1+\lambda) y(x,t+1) - \frac{-\lambda}{2} y(x+1,t+1) = \frac{\lambda}{2} y(x-1,t) + (1-\lambda) y(x,t) + \frac{\lambda}{2}y(x+1,t)
$$

Matricialmente esto lo podemos pensar como:

$$
A\,y^{n+1} = B\,y^{n} + d,
$$

donde:

$$
A =
\begin{bmatrix}
1 + \lambda & -\frac{\lambda}{2} & 0 & \dots & 0 \\
-\frac{\lambda}{2} & 1 + \lambda & -\frac{\lambda}{2} & \ddots & \vdots \\
0 & \ddots & \ddots & \ddots & 0 \\
\vdots & \ddots & -\frac{\lambda}{2} & 1 + \lambda & -\frac{\lambda}{2} \\
0 & \dots & 0 & -\frac{\lambda}{2} & 1 + \lambda
\end{bmatrix},
$$

$$
B =
\begin{bmatrix}
1 - \lambda & \frac{\lambda}{2} & 0 & \dots & 0 \\
\frac{\lambda}{2} & 1 - \lambda & \frac{\lambda}{2} & \ddots & \vdots \\
0 & \ddots & \ddots & \ddots & 0 \\
\vdots & \ddots & \frac{\lambda}{2} & 1 - \lambda & \frac{\lambda}{2} \\
0 & \dots & 0 & \frac{\lambda}{2} & 1 - \lambda
\end{bmatrix},
$$

$$
d = \frac{\lambda}{2}
\begin{bmatrix}
y_0^{n+1} + y_0^{n} \\
0 \\
\vdots \\
0 \\
y_{N_x}^{n+1} + y_{N_x}^{n}
\end{bmatrix}.
$$

In [16]:
N = Nx 

# Armado de matrices A y B

main_A = (1.0 + lam)   * np.ones(N-1)
off_A  = (-0.5 * lam)  * np.ones(N-2)
A_CN   = np.diag(main_A) + np.diag(off_A, 1) + np.diag(off_A, -1)

main_B = (1.0 - lam)   * np.ones(N-1)
off_B  = ( 0.5 * lam)  * np.ones(N-2)
B_CN   = np.diag(main_B) + np.diag(off_B, 1) + np.diag(off_B, -1)

for n in range(Nt):
    # Fronteras en tiempos n y n+1
    yL_n   = frontera_izquierda(tau[n])
    yR_n   = frontera_derecha(tau[n])
    yL_np1 = frontera_izquierda(tau[n+1])
    yR_np1 = frontera_derecha(tau[n+1])

    # Lado derecho
    y_int_n = y[1:-1].copy()               
    rhs = B_CN @ y_int_n 

    # vector de borde (solo extremos interior)
    rhs[0]  += 0.5 * lam * (yL_n   + yL_np1)
    rhs[-1] += 0.5 * lam * (yR_n   + yR_np1)

    # Nuevamente usamos el metodo de numpy para resolver el sistema lineal
    y_int_np1 = np.linalg.solve(A_CN, rhs)

    # Reconstruir solución completa en n+1
    y[0]      = yL_np1
    y[1:-1]   = y_int_np1
    y[-1]     = yR_np1

# Nuevamente usamos el metodo de comparación con el valor exacto para S0=K como en el explícito e implícito.

shift = (2.0*r/sigma**2 - 1.0) * tau_max
S_grid = np.exp(x - shift)
V_num = y.copy()  # en t=0, V(S,0) = y(x, τ_max)
S0 = K
idx = np.argmin(np.abs(S_grid - S0))
V_S0_nearest = V_num[idx]
print(f"[Crank–Nicolson] Precio numérico para S0={S0}: {V_S0_nearest:.6f}")

[Crank–Nicolson] Precio numérico para S0=100.0: 10.490603


### Chequeo con un función de una librería

In [26]:
from py_vollib.black_scholes import black_scholes
from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega, theta

S = 100
K = 100
r = 0.05
sigma = 0.20
t = 1.0
flag = 'c'   # 'c' para call, 'p' para put

price = black_scholes(flag, S, K, t, r, sigma)
print("Precio B-S:", price)
print("Delta:", delta(flag, S, K, t, r, sigma))
print("Gamma:", gamma(flag, S, K, t, r, sigma))

Precio B-S: 10.450583572185561
Delta: 0.6368306511756191
Gamma: 0.018762017345846895


In [ ]:
for i in range(5):
    print("Hola, mundo!", i)
 
x = 10

Hola, mundo! 0
Hola, mundo! 1
Hola, mundo! 2
Hola, mundo! 3
Hola, mundo! 4


In [ ]:
for i in range(5):
    print("Hola, mundo!", i)
    